In [1]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json
import requests

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,processDatesIT

In [2]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [3]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [4]:
! rm ../../Finances.db
! rm schema.sql

In [5]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [6]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()
schemas = open("schema.sql", "a+")

In [7]:
%%time
for name, url in independents.iteritems():
    print(name,url)
    tab = get_appended(url,apikey,name)
    a = pd.io.sql.get_schema(tab,name)
    c.execute('drop table if exists '+ name)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(name,con=conn,index=False,if_exists='append')

#setup database
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = ['000'+str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs_c + indexes + ciks + funds + commods))

#setup subset of tickers
dow_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_DOWJLIST;',con=conn).symbol)
nasdaq_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_NASDAQLIST;',con=conn).symbol)
sp500_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

#define subset to work with
stocksub = dow_tickers
#stocksub = sp500_tickers
indexsub = ['^IXIC','^GSPC','^DJI']
#indexsub = ['^IXIC','^GSPC','^DJI','^VIX','^RUA','^FTSE','^NDX','^N225','GDAXI']

#get top US etfs proportionate to stock list
num_etfs = int(np.ceil(len(stocksub)/10))
profile = get_appended(dependents['P1b1000_CV_PROFILE'],apikey,'P1b1000_CV_PROFILE',tickers = etfs_c)
etfsub = list(profile[profile.country == 'US'].sort_values(by='volAvg',ascending=False).symbol[:num_etfs+1])
tickersub = stocksub + etfsub

#get mutual fund data
fundtable = get_appended(dependents['T1_INST_MUTFUNDHOLDERS'],apikey,'T1_INST_MUTFUNDHOLDERS',tickers=tickersub,verbose=False)
fundsub = list(fundtable.holder.value_counts().index)

mfs = pd.io.sql.read_sql('select * from G1_MF_SYMBOLS;',con=conn)
fundsub = list(np.random.choice(list(mfs[mfs.name.isin(fundsub)].symbol),int(np.ceil(len(stocksub)/3))))

#establish subset of commods
#commodsub = commods[:3]
commodsub = commods

#get ciks of institutional holders for ciksub
instable = get_appended(dependents['T1_INST_HOLDERS'],apikey,'T1_INST_HOLDERS',tickers=tickersub)
instsub = list(instable.holder.value_counts().index)
instsub = list(np.random.choice(instsub,500)) #account for many missing inst-cik mappings

#retrieve ciks
ciklist = pd.io.sql.read_sql('select * from G1_INST_CIKLIST',con=conn)
ciksub = list(ciklist[ciklist.name.apply(lambda x: x.lower()).isin([i.lower() for i in instsub])].cik)
ciksub = ['000'+str(c) for c in (ciksub)]
ciksub = [c for c in ciksub if len(c)==10]

#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocksub + etfsub + commodsub + fundsub,
     'P':stocksub + etfsub + fundsub,
     'T':stocksub + etfsub,
     'S':stocksub,
     'E':etfsub,
     'C':commodsub,
     'F':ciksub,
     'I':indexsub,
     'M':fundsub}

#dependentsub = dependents.iloc[43:] #for testing specific urls

##%%time
schemas = open("schema.sql", "a+")
for name, url in dependents.iteritems():
    
    tablename = '_'.join(name.split('_')[1:])
    print(tablename)
    
    if name[0] == 'U':
            continue 
    elif name[0]=='G': #if ticker info not needed
        continue
    else:
        tab = get_appended(url,apikey,name,tickers=d.get(name[0]),verbose=True)
    
    if len(tab) == 0: 
        print(tablename,' FAILED')
        #log error
        continue
    
    #load sql
    a = pd.io.sql.get_schema(tab,tablename)
    c.execute('drop table if exists '+ tablename)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(tablename,con=conn,index=False,if_exists='append')

G1_CV_SYMBOLS https://financialmodelingprep.com/api/v3/stock/list?apikey=
G1_CV_ETFS https://financialmodelingprep.com/api/v3/etf/list?apikey=
G1_CV_TRADABLE https://financialmodelingprep.com/api/v3/available-traded/list?apikey=
G1_CV_DELISTED https://financialmodelingprep.com/api/v3/delisted-companies?limit=1000000000&apikey=
G1_CAL_EARNINGSCALENDAR_UPC https://financialmodelingprep.com/api/v3/earning_calendar?apikey=
G1_INST_CIKLIST https://financialmodelingprep.com/api/v3/cik_list?apikey=
G1_MI_MAJORINDEXES https://financialmodelingprep.com/api/v3/quotes/index?apikey=
G1_MI_SP500LIST https://financialmodelingprep.com/api/v3/sp500_constituent?apikey=
G1_MI_SP500HIST https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey=
G1_MI_NASDAQLIST https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=
G1_MI_NASDAQHISTLIST https://financialmodelingprep.com/api/v3/historical/nasdaq_constituent?apikey=
G1_MI_DOWJLIST https://financialmodelingprep.com/api/v3/dowj

/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2788: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


CV_DCF_YR


/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2788: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


CV_DCF_QTR


/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2788: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


CV_DCF_DAY
CV_MARKETCAP
CV_MARKETCAP_DAY
CV_NEWS
Running Batch: CRM,WBA,V,NKE,UNH,TRV,VZ,INTC,WMT,JNJ,DIS,MCD,JPM,CAT,BA,AMGN,DOW,AAPL,GS,CSCO,MSFT,HD,PG,MRK,IBM,HON,KO,CVX,AXP,MMM,UVXY,SPY,SQQQ,VXX
CV_PRESSRELEASES
INS_STOCKTRADING
CAL_EARNINGSCALENDAR_HIST
INST_HOLDERS
INST_MUTFUNDHOLDERS
INST_ETFHOLDERS
INST_ETFSECTORWEIGHTS
INST_ETFCOUNTRYWEIGHTS
INST_13F
TS_PRICE_DAY
TS_DIVIDENDS_HIST
TS_SPLITS_HIST
MI_INDEXPRICEHIST_DAY
COM_PRICES_DAY
MF_PRICES_DAY
CPU times: user 3min 22s, sys: 5.31 s, total: 3min 28s
Wall time: 17min 20s
